In [22]:
import json
import pandas as pd
import numpy as np
from pprint import pprint
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, linregress
import re
sns.set_style('whitegrid')
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 100)

In [23]:
with open('data2.json') as data_file:    
    data = json.load(data_file)

In [24]:
for i, j in data.items():
    pprint(i)

'game-settings'
'phases'
'events'
'teams'
'element_types'
'total-players'
'elements'


In [25]:
data['game-settings']

{'element_type': {'1': {'bps_clean_sheets': 12,
   'bps_goals_scored': 12,
   'scoring_clean_sheets': 4,
   'scoring_goals_conceded': -1,
   'scoring_goals_scored': 6,
   'squad_max_play': 1,
   'squad_min_play': 1,
   'squad_select': 2,
   'sub_positions_locked': [12],
   'ui_shirt_specific': True},
  '2': {'bps_clean_sheets': 12,
   'bps_goals_scored': 12,
   'scoring_clean_sheets': 4,
   'scoring_goals_conceded': -1,
   'scoring_goals_scored': 6,
   'squad_max_play': 5,
   'squad_min_play': 3,
   'squad_select': 5},
  '3': {'bps_clean_sheets': 0,
   'bps_goals_scored': 18,
   'scoring_clean_sheets': 1,
   'scoring_goals_conceded': 0,
   'scoring_goals_scored': 5,
   'squad_max_play': 5,
   'squad_min_play': 2,
   'squad_select': 5},
  '4': {'bps_clean_sheets': 0,
   'bps_goals_scored': 24,
   'scoring_clean_sheets': 0,
   'scoring_goals_conceded': 0,
   'scoring_goals_scored': 4,
   'squad_max_play': 3,
   'squad_min_play': 1,
   'squad_select': 3}},
 'game': {'bps_assists': 9,
  'b

In [26]:
player_data_json = data['elements']

In [27]:
player_data_df = pd.DataFrame(player_data_json)

In [28]:
pdata = player_data_df.copy().fillna(0)

In [29]:
pdata.head(3)

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,dreamteam_count,ea_index,element_type,ep_next,ep_this,event_points,first_name,form,goals_conceded,goals_scored,ict_index,id,in_dreamteam,influence,loaned_in,loaned_out,loans_in,loans_out,minutes,news,now_cost,own_goals,penalties_missed,penalties_saved,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,squad_number,status,team,team_code,threat,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards
0,0,0,0,0.0,0.0,0,48844,0,0,0,0,0.0,0,0,1,0.0,1.0,0,David,0.0,0,0,0.0,1,False,0.0,0,0,0,0,0,,50,0,0,0,48844.jpg,0.0,0,0,Ospina,0.2,False,13.0,a,1,3,0.0,0,249,54,1876,553,0.0,0.0,Ospina,0
1,0,0,23,0.0,0.0,0,11334,0,0,0,0,0.0,0,0,1,2.0,3.0,3,Petr,2.0,4,0,3.8,2,False,38.0,0,0,0,0,180,,55,0,0,0,11334.jpg,2.0,0,3,Cech,5.7,False,33.0,a,1,3,0.0,4,5180,843,25057,8311,0.4,0.7,Cech,0
2,0,0,0,0.0,0.0,0,98980,0,0,0,0,0.0,0,0,1,0.0,0.0,0,Damian Emiliano,0.0,0,0,0.0,3,False,0.0,0,0,0,0,0,Season-long loan to Getafe,40,0,0,0,98980.jpg,0.0,0,0,Martinez,0.9,False,26.0,u,1,3,0.0,0,156,45,7403,1827,0.0,0.0,Martinez,0


In [30]:
to_drop = ['chance_of_playing_this_round','chance_of_playing_next_round','code','cost_change_event','cost_change_event_fall','cost_change_start','cost_change_start_fall','dreamteam_count','ea_index','ep_this','event_points','form','ict_index','in_dreamteam','loaned_in','loaned_out','loans_in','loans_out','news','photo','special','squad_number','status','transfers_in','transfers_in_event','transfers_out','transfers_out_event','value_form','value_season']

In [31]:
pdata.drop(to_drop, axis=1, inplace = True)

In [32]:
pdata.columns

Index(['assists', 'bonus', 'bps', 'clean_sheets', 'creativity', 'element_type',
       'ep_next', 'first_name', 'goals_conceded', 'goals_scored', 'id',
       'influence', 'minutes', 'now_cost', 'own_goals', 'penalties_missed',
       'penalties_saved', 'points_per_game', 'red_cards', 'saves',
       'second_name', 'selected_by_percent', 'team', 'team_code', 'threat',
       'total_points', 'web_name', 'yellow_cards'],
      dtype='object')

In [33]:
pdata['full_name'] = pdata.first_name + " " + pdata.second_name

In [34]:
pdata['element_type_name'] = pdata.element_type.map({x['id']:x['singular_name_short'] for x in data['element_types']})

In [35]:
pdata = pdata.loc[:,['full_name','first_name','second_name', 'element_type','element_type_name','id','team', 'team_code', 'web_name',
                     'saves','penalties_saved','clean_sheets','goals_conceded',
                     'bonus', 'bps','creativity','ep_next','influence', 'threat',
                     'goals_scored','assists','minutes', 'own_goals',
                     'yellow_cards', 'red_cards','penalties_missed',
                     'selected_by_percent', 'now_cost','points_per_game','total_points']]

In [36]:
pdata['team'] = pdata.team.map({x['id']:x['name'] for x in data['teams']})

In [37]:
pdata.head()

,full_name,first_name,second_name,element_type,element_type_name,id,team,team_code,web_name,saves,penalties_saved,clean_sheets,goals_conceded,bonus,bps,creativity,ep_next,influence,threat,goals_scored,assists,minutes,own_goals,yellow_cards,red_cards,penalties_missed,selected_by_percent,now_cost,points_per_game,total_points
0,David Ospina,David,Ospina,1,GKP,1,Arsenal,3,Ospina,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0.2,50,0.0,0
1,Petr Cech,Petr,Cech,1,GKP,2,Arsenal,3,Cech,3,0,0,4,0,23,0.0,2.0,38.0,0.0,0,0,180,0,0,0,0,5.7,55,2.0,4
2,Damian Emiliano Martinez,Damian Emiliano,Martinez,1,GKP,3,Arsenal,3,Martinez,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0.9,40,0.0,0
3,Laurent Koscielny,Laurent,Koscielny,2,DEF,4,Arsenal,3,Koscielny,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0.6,60,0.0,0
4,Per Mertesacker,Per,Mertesacker,2,DEF,5,Arsenal,3,Mertesacker,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0.5,50,0.0,0


In [38]:
pdata.corr()['total_points']

element_type        0.017879
id                  0.009693
team_code          -0.031238
saves               0.204554
penalties_saved          NaN
clean_sheets        0.730337
goals_conceded      0.269291
bonus               0.741820
bps                 0.908887
goals_scored        0.589466
assists             0.482411
minutes             0.714785
own_goals          -0.049867
yellow_cards        0.148148
red_cards          -0.073979
penalties_missed         NaN
now_cost            0.311936
total_points        1.000000
Name: total_points, dtype: float64

In [39]:
pdata.pivot_table(index='element_type_name', values='total_points', aggfunc=np.mean)

element_type_name
DEF    3.405714
FWD    4.098592
GKP    3.244898
MID    2.868778
Name: total_points, dtype: float64

In [40]:
pdata.element_type_name.value_counts()

MID    221
DEF    175
FWD     71
GKP     49
Name: element_type_name, dtype: int64

In [41]:
pdata.columns

Index(['full_name', 'first_name', 'second_name', 'element_type',
       'element_type_name', 'id', 'team', 'team_code', 'web_name', 'saves',
       'penalties_saved', 'clean_sheets', 'goals_conceded', 'bonus', 'bps',
       'creativity', 'ep_next', 'influence', 'threat', 'goals_scored',
       'assists', 'minutes', 'own_goals', 'yellow_cards', 'red_cards',
       'penalties_missed', 'selected_by_percent', 'now_cost',
       'points_per_game', 'total_points'],
      dtype='object')

In [42]:
pdata.element_type_name.value_counts()

MID    221
DEF    175
FWD     71
GKP     49
Name: element_type_name, dtype: int64

# Pulp implementation

In [43]:
from pulp import *

In [44]:
prob = pulp.LpProblem('FantasyTeam', pulp.LpMaximize)

In [45]:
decision_variables = []
for rownum, row in pdata.iterrows():
    variable = str('x' + str(rownum))
    variable = pulp.LpVariable(str(variable), lowBound = 0, upBound = 1, cat= 'Integer') #make variables binary
    decision_variables.append(variable)

print ("Total number of decision_variables: " + str(len(decision_variables)))

Total number of decision_variables: 516


In [46]:
total_points = ""
for rownum, row in pdata.iterrows():
    for i, schedule in enumerate(decision_variables):
        if rownum == i:
            formula = row['total_points']*schedule
            total_points += formula

prob += total_points
print ("Optimization function: " + str(total_points))

Optimization function: 4*x1 + x10 + 10*x100 + 2*x101 + x102 - x104 + 9*x105 + 6*x106 + 4*x107 + 4*x108 + 2*x11 + 4*x110 + 4*x111 + 11*x114 + 3*x116 + 3*x117 + x118 + 7*x12 + x122 + 2*x123 + 2*x124 + 2*x125 + x126 + 3*x130 + 3*x131 + 3*x133 + 10*x134 + 8*x137 + 9*x139 + 2*x14 + 9*x142 + 2*x143 + 10*x145 + x147 + 4*x15 + 3*x150 + x151 + 2*x153 + 5*x154 + 3*x155 + 2*x157 + 10*x158 + 2*x159 + 17*x160 + 17*x162 + 15*x163 + 12*x164 + 6*x165 + x166 + 8*x17 + 16*x170 + 16*x171 + 4*x172 + 5*x174 + 4*x18 + 9*x180 + 6*x181 + 2*x182 + x183 + 5*x185 + 13*x187 + 8*x188 + 4*x190 + 6*x191 + 6*x192 + 18*x197 + 16*x198 + 8*x199 + x20 + 2*x202 + x203 + 5*x205 + 15*x208 + 5*x21 + 13*x210 + x211 + 6*x212 + 2*x214 + x215 + 7*x217 + 7*x218 + 7*x219 + 12*x22 + 7*x221 + 4*x224 + 5*x226 + 14*x229 + 5*x231 + 12*x233 + 14*x234 + 2*x235 + x237 + x238 + 8*x239 + 6*x24 + 8*x241 + 7*x243 + 10*x244 + 6*x246 + 10*x247 + 8*x249 + 8*x25 + 5*x251 + 3*x254 + 2*x255 + 11*x256 + x257 + 2*x258 + 12*x259 + 12*x263 + 11*x266 + 

In [47]:
avail_cash = 264
total_paid = ""
for rownum, row in pdata.iterrows():
    for i, schedule in enumerate(decision_variables):
        if rownum == i:
            formula = row['now_cost']*schedule
            total_paid += formula

prob += (total_paid == avail_cash)

In [48]:
avail_gk = 1
total_gk = ""
for rownum, row in pdata.iterrows():
    for i, schedule in enumerate(decision_variables):
        if rownum == i:
            if row['element_type_name'] == 'GKP':
                formula = 1*schedule
                total_gk += formula

prob += (total_gk == avail_gk)
print(total_gk)

x0 + x1 + x114 + x115 + x134 + x135 + x136 + x161 + x162 + x188 + x189 + x2 + x212 + x213 + x239 + x240 + x259 + x260 + x261 + x28 + x285 + x286 + x29 + x30 + x304 + x305 + x327 + x328 + x349 + x350 + x374 + x375 + x395 + x396 + x419 + x420 + x437 + x438 + x468 + x472 + x478 + x505 + x52 + x53 + x73 + x74 + x93 + x94 + x95


In [49]:
avail_def = 1
total_def = ""
for rownum, row in pdata.iterrows():
    for i, schedule in enumerate(decision_variables):
        if rownum == i:
            if row['element_type_name'] == 'DEF':
                formula = 1*schedule
                total_def += formula

prob += (total_def == avail_def)
print(len(total_def))

175


In [50]:
avail_mid = 2
total_mid = ""
for rownum, row in pdata.iterrows():
    for i, schedule in enumerate(decision_variables):
        if rownum == i:
            if row['element_type_name'] == 'MID':
                formula = 1*schedule
                total_mid += formula

prob += (total_mid == avail_mid)
print(len(total_mid))

221


In [82]:
# avail_fwd = 1
# total_fwd = ""
# for rownum, row in pdata.iterrows():
#     for i, schedule in enumerate(decision_variables):
#         if rownum == i:
#             if row['element_type_name'] == 'FWD':
#                 formula = 1*schedule
#                 total_fwd += formula

# prob += (total_fwd == avail_fwd)
# print(len(total_fwd))

70


In [51]:
team_dict= {}
for team in set(pdata.team_code):
    team_dict[str(team)]=dict()
    team_dict[str(team)]['avail'] = 3
    team_dict[str(team)]['total'] = ""
    for rownum, row in pdata.iterrows():
        for i, schedule in enumerate(decision_variables):
            if rownum == i:
                if row['team_code'] == team:
                    formula = 1*schedule
                    team_dict[str(team)]['total'] += formula

    prob += (team_dict[str(team)]['total'] <= team_dict[str(team)]['avail'])
print(len(team_dict))

20


In [52]:
print(prob)

FantasyTeam:
MAXIMIZE
4*x1 + 1*x10 + 10*x100 + 2*x101 + 1*x102 + -1*x104 + 9*x105 + 6*x106 + 4*x107 + 4*x108 + 2*x11 + 4*x110 + 4*x111 + 11*x114 + 3*x116 + 3*x117 + 1*x118 + 7*x12 + 1*x122 + 2*x123 + 2*x124 + 2*x125 + 1*x126 + 3*x130 + 3*x131 + 3*x133 + 10*x134 + 8*x137 + 9*x139 + 2*x14 + 9*x142 + 2*x143 + 10*x145 + 1*x147 + 4*x15 + 3*x150 + 1*x151 + 2*x153 + 5*x154 + 3*x155 + 2*x157 + 10*x158 + 2*x159 + 17*x160 + 17*x162 + 15*x163 + 12*x164 + 6*x165 + 1*x166 + 8*x17 + 16*x170 + 16*x171 + 4*x172 + 5*x174 + 4*x18 + 9*x180 + 6*x181 + 2*x182 + 1*x183 + 5*x185 + 13*x187 + 8*x188 + 4*x190 + 6*x191 + 6*x192 + 18*x197 + 16*x198 + 8*x199 + 1*x20 + 2*x202 + 1*x203 + 5*x205 + 15*x208 + 5*x21 + 13*x210 + 1*x211 + 6*x212 + 2*x214 + 1*x215 + 7*x217 + 7*x218 + 7*x219 + 12*x22 + 7*x221 + 4*x224 + 5*x226 + 14*x229 + 5*x231 + 12*x233 + 14*x234 + 2*x235 + 1*x237 + 1*x238 + 8*x239 + 6*x24 + 8*x241 + 7*x243 + 10*x244 + 6*x246 + 10*x247 + 8*x249 + 8*x25 + 5*x251 + 3*x254 + 2*x255 + 11*x256 + 1*x257 + 2*x25

In [53]:
prob.writeLP('FantasyTeam.lp')

In [54]:
optimization_result = prob.solve()

In [55]:
assert optimization_result == pulp.LpStatusOptimal
print("Status:", LpStatus[prob.status])
print("Optimal Solution to the problem: ", value(prob.objective))
print ("Individual decision_variables: ")
for v in prob.variables():
	print(v.name, "=", v.varValue)

Status: Optimal
Optimal Solution to the problem:  83.0
Individual decision_variables: 
x0 = 0.0
x1 = 0.0
x10 = 0.0
x100 = 0.0
x101 = 0.0
x102 = 0.0
x103 = 0.0
x104 = 0.0
x105 = 0.0
x106 = 0.0
x107 = 0.0
x108 = 0.0
x109 = 0.0
x11 = 0.0
x110 = 0.0
x111 = 0.0
x112 = 0.0
x113 = 0.0
x114 = 0.0
x115 = 0.0
x116 = 0.0
x117 = 0.0
x118 = 0.0
x119 = 0.0
x12 = 0.0
x120 = 0.0
x121 = 0.0
x122 = 0.0
x123 = 0.0
x124 = 0.0
x125 = 0.0
x126 = 0.0
x127 = 0.0
x128 = 0.0
x129 = 0.0
x13 = 0.0
x130 = 0.0
x131 = 0.0
x132 = 0.0
x133 = 0.0
x134 = 0.0
x135 = 0.0
x136 = 0.0
x137 = 0.0
x138 = 0.0
x139 = 0.0
x14 = 0.0
x140 = 0.0
x141 = 0.0
x142 = 0.0
x143 = 0.0
x144 = 0.0
x145 = 0.0
x146 = 0.0
x147 = 0.0
x148 = 0.0
x149 = 0.0
x15 = 0.0
x150 = 0.0
x151 = 0.0
x152 = 0.0
x153 = 0.0
x154 = 0.0
x155 = 0.0
x156 = 0.0
x157 = 0.0
x158 = 0.0
x159 = 0.0
x16 = 0.0
x160 = 0.0
x161 = 0.0
x162 = 1.0
x163 = 0.0
x164 = 0.0
x165 = 0.0
x166 = 0.0
x167 = 0.0
x168 = 0.0
x169 = 0.0
x17 = 0.0
x170 = 0.0
x171 = 1.0
x172 = 0.0
x173 = 0.0
x

In [56]:
variable_name = []
variable_value = []

for v in prob.variables():
    variable_name.append(v.name)
    variable_value.append(v.varValue)

df = pd.DataFrame({'variable': variable_name, 'value': variable_value})
for rownum, row in df.iterrows():
    value = re.findall(r'(\d+)', row['variable'])
    df.loc[rownum, 'variable'] = int(value[0])

df = df.sort_index(by='variable')

#append results
for rownum, row in pdata.iterrows():
    for results_rownum, results_row in df.iterrows():
        if rownum == results_row['variable']:
            pdata.loc[rownum, 'decision'] = results_row['value']
            
pdata.head()

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:13: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)


,full_name,first_name,second_name,element_type,element_type_name,id,team,team_code,web_name,saves,penalties_saved,clean_sheets,goals_conceded,bonus,bps,creativity,ep_next,influence,threat,goals_scored,assists,minutes,own_goals,yellow_cards,red_cards,penalties_missed,selected_by_percent,now_cost,points_per_game,total_points,decision
0,David Ospina,David,Ospina,1,GKP,1,Arsenal,3,Ospina,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0.2,50,0.0,0,0.0
1,Petr Cech,Petr,Cech,1,GKP,2,Arsenal,3,Cech,3,0,0,4,0,23,0.0,2.0,38.0,0.0,0,0,180,0,0,0,0,5.7,55,2.0,4,0.0
2,Damian Emiliano Martinez,Damian Emiliano,Martinez,1,GKP,3,Arsenal,3,Martinez,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0.9,40,0.0,0,0.0
3,Laurent Koscielny,Laurent,Koscielny,2,DEF,4,Arsenal,3,Koscielny,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0.6,60,0.0,0,0.0
4,Per Mertesacker,Per,Mertesacker,2,DEF,5,Arsenal,3,Mertesacker,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0.5,50,0.0,0,0.0


In [57]:
pdata[pdata.decision==1].sort_values('element_type')

,full_name,first_name,second_name,element_type,element_type_name,id,team,team_code,web_name,saves,penalties_saved,clean_sheets,goals_conceded,bonus,bps,creativity,ep_next,influence,threat,goals_scored,assists,minutes,own_goals,yellow_cards,red_cards,penalties_missed,selected_by_percent,now_cost,points_per_game,total_points,decision
162,Jonas Lössl,Jonas,Lössl,1,GKP,163,Huddersfield,38,Lössl,9,0,2,0,3,63,0.0,8.0,68.8,0.0,0,0,180,0,0,0,0,3.9,45,8.5,17,1.0
467,Ahmed El-Sayed Hegazi,Ahmed El-Sayed,Hegazi,2,DEF,468,West Brom,35,Hegazi,0,0,2,0,5,63,10.4,11.5,82.2,53.0,1,0,180,0,0,0,0,17.3,47,11.5,23,1.0
22,Granit Xhaka,Granit,Xhaka,3,MID,23,Arsenal,3,Xhaka,0,0,0,4,2,49,103.6,6.0,65.6,39.0,0,2,168,0,0,0,0,16.7,56,6.0,12,1.0
171,Aaron Mooy,Aaron,Mooy,3,MID,172,Huddersfield,38,Mooy,0,0,2,0,2,57,56.1,7.5,97.8,24.0,1,1,180,0,0,0,0,5.3,56,8.0,16,1.0
90,Sam Vokes,Sam,Vokes,4,FWD,91,Burnley,90,Vokes,0,0,0,3,3,58,26.0,6.0,65.2,70.0,2,0,180,0,0,0,0,6.5,60,7.5,15,1.0


In [58]:
pdata[pdata.decision==1].now_cost.sum()

264

In [59]:
pdata[pdata.decision==1].total_points.sum()

83